# Lezione 5

In [ ]:
from liblet import Derivation, Grammar, animate_derivation

# Parsing

## Top down

In [ ]:
w = 'aabbcc'

G = Grammar.from_string("""
S -> a S Q | a b c
b Q c -> b b c c  
c Q -> Q c
""", False)

In [ ]:
animate_derivation(Derivation(G).step([
  (0, 0), (1, 1), (3, 3), (2, 2)
]), height = '350px')

interactive(children=(IntSlider(value=0, description='n', max=4), Output(layout=Layout(height='350px'))), _dom…

## Bottom up

Nel caso *bottom up* si può procedere con una grammatica ottenuta rovesciando le produzioni (e aggiungendo una produzione `Inizio`$\to \omega$ dove `Inizio` è il nuovo simbolo di partenza (e volendo una produzione $S\to $`Fine` dove `Fine` è un simbolo usato per indicare la fine della derivazione).

In [ ]:
GR = Grammar.from_string("""
Inizio -> a a b b c c
a S Q -> S
a b c -> S 
b b c c -> b Q c
Q c -> c Q
S -> Fine
""", False)

In [ ]:
animate_derivation(Derivation(GR).step([
  (0, 0), (3, 2), (4, 3), (2, 1), (1, 0), (5, 0)
]), height = '500px')

interactive(children=(IntSlider(value=0, description='n', max=6), Output(layout=Layout(height='500px'))), _dom…